In [ ]:
from lorenz_model import generate_lorenz_hi_dim
import numpy as np

# ----- 1. Generate data like in the paper -----
rng = np.random.default_rng(0)
dt = 0.02

t, X_train, Z_train, U = generate_lorenz_hi_dim(
    n_traj=2048,     # training trajectories
    dt=dt,
    t_final=5.0,
    n_spatial=128,
    seed=0,
)

print("X_train shape:", X_train.shape)   # (2048, 250, 128)

KeyboardInterrupt: 

In [4]:
from model import SINDy_Autoencoder, TimeSeriesDataset

x_dim = X_train.shape[-1]   # 128
z_dim = 3                   # as in the SI table

model = SINDy_Autoencoder(
                        x_dim=x_dim,
                        z_dim=z_dim,
                        enc_hidden=(64, 32),    # SI for Lorenz: encoder widths 64, 32
                        dec_hidden=(32, 64),
                        poly_order=3,
                        include_bias=True,
)

In [6]:
import torch
from trainer import Trainer_SINDyAE

device = "cuda" if torch.cuda.is_available() else "cpu"

trainer = Trainer_SINDyAE(
    model=model,
    dt=dt,
    diff_method="finite-diff",   # or your tvdiff
    diff_kwargs={},
    batch_size=8000,        # SI uses batch_size=8000 for Lorenz
    device=device,
    lambda_recon=1.0,
    lambda_dx=1e-4,         # λ1 from Table S1
    lambda_dz=0.0,          # λ2 from Table S1
    threshold=0.1,
    threshold_freq=500,
    lr=1e-3,
)

trainer.fit(
    X_train,
    n_epochs=10_000,
    log_every=500,
)

ValueError: operands could not be broadcast together with shapes (2048,248,128) (2048,2,128) 